![test](../notebooks/images/header3.png)

# Workshop goals
* Feedforward networks and backpropagation
* Activation functions
* Deep learning
* 

# How to get and open this file:
1. `git pull`
1. `docker-compose build`
1. `docker-compose up`
1. Open `localhost:8888` in a browser and paste in the token.

# To stop docker:
1. `control-c` to stop Jupyter.
1. `docker-compose down`